In [1]:
import nltk
nltk.download('wordnet')
import random
from transformers import pipeline
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2025-08-04 12:44:39.847010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754311480.026866      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754311480.084304      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
try:
    print("Loading sentiment analysis model...")
    sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    print("Model loaded successfully.\n")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have an internet connection to download the model.")
    exit()

Loading sentiment analysis model...


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded successfully.



In [6]:
def get_sentiment(text):
    """
    Analyzes the sentiment of a given text using the loaded pipeline.
    Returns the predicted label and score.
    """
    result = sentiment_pipeline(text)[0]
    return result['label'], result['score']


def find_synonyms(word):
    """
    Finds synonyms for a given word using WordNet.
    Returns a list of unique synonym words.
    """
    synonyms = set()
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            synonym = lemma.name().replace('_', ' ')
            # Exclude the original word and only consider single-word synonyms
            if synonym != word and ' ' not in synonym:
                synonyms.add(synonym)
    return list(synonyms)


def perform_adversarial_attack(original_text):
    """
    Performs a simple synonym replacement attack on the text.
    The goal is to flip the sentiment from POSITIVE to NEGATIVE while
    maintaining a coherent sentence structure.
    """
    print("Starting adversarial attack...")

    # Get the original sentiment
    original_label, original_score = get_sentiment(original_text)
    print(f"Original Text: '{original_text}'")
    print(f"Original Sentiment: {original_label} (Score: {original_score:.4f})\n")

    # Only proceed if the original sentiment is positive
    if original_label != 'POSITIVE':
        print("Original text is not positive. The attack is designed to flip from POSITIVE to NEGATIVE.")
        return

    words = original_text.split()
    
    # Iterate through the words in their original order
    for i, word in enumerate(words):
        synonyms = find_synonyms(word)

        if synonyms:
            print(f"Attempting to replace '{word}' with a synonym...")
            # Try to replace with a random synonym
            random.shuffle(synonyms)
            for synonym in synonyms:
                print(f"  - Trying synonym: '{synonym}'")
                # Create a new sentence by replacing the word at its original position
                perturbed_words = list(words)
                perturbed_words[i] = synonym
                perturbed_text = ' '.join(perturbed_words)

                # Get the sentiment of the perturbed text
                perturbed_label, perturbed_score = get_sentiment(perturbed_text)
                print(perturbed_text)
                print(perturbed_label)
                print(perturbed_score)

                if perturbed_label != original_label:
                    print("-" * 50)
                    print("ATTACK SUCCESSFUL")
                    print(f"Perturbed Text: '{perturbed_text}'")
                    print(f"Flipped Sentiment: {perturbed_label} (Score: {perturbed_score:.4f})")
                    print(f"The word '{word}' was replaced with '{synonym}'.")
                    print("-" * 50)
                    return

    print("\nAttack failed. Could not find a synonym replacement to flip the sentiment.")

In [7]:
target_text = "The acting was absolutely outstanding."

perform_adversarial_attack(target_text)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Starting adversarial attack...
Original Text: 'The acting was absolutely outstanding.'
Original Sentiment: POSITIVE (Score: 0.9999)

Attempting to replace 'acting' with a synonym...
  - Trying synonym: 'dissemble'
The dissemble was absolutely outstanding.
POSITIVE
0.999872088432312
  - Trying synonym: 'roleplay'
The roleplay was absolutely outstanding.
POSITIVE
0.9998650550842285
  - Trying synonym: 'playacting'
The playacting was absolutely outstanding.
POSITIVE
0.9998751878738403
  - Trying synonym: 'play'
The play was absolutely outstanding.
POSITIVE
0.9998695850372314
  - Trying synonym: 'work'
The work was absolutely outstanding.
POSITIVE
0.9998644590377808
  - Trying synonym: 'act'
The act was absolutely outstanding.
POSITIVE
0.9998700618743896
  - Trying synonym: 'pretend'
The pretend was absolutely outstanding.
POSITIVE
0.9997398257255554
  - Trying synonym: 'move'
The move was absolutely outstanding.
POSITIVE
0.9998646974563599
  - Trying synonym: 'represent'
The represent was